<a href="https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/6_Collaborative_Based_Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Table of Contents

Note: The internal links work in Google Colab.

1. **[Preface](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/MovieLens.ipynb#preface)**
2. **[Introduction](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/MovieLens.ipynb#introduction)**
3. **[Exploratory Data Analysis](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/3_Exploratory_Data_Analysis.ipynb.ipynb#eda)**
4. **[Framework](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/4_Framework.ipynb#framework)**
5. **[Content Based Recommenders](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/5_Content_Based_Recommenders.ipynb#content)**
5. **[Collaborative Based Recommenders](#collaborative)**
    - 5.1 - [Introduction](#introduction)
    - 5.2 - [ContentKNNAlgorithm.py](#contentknnalgo)
    - 5.3 - [Import Files](#import)
    - 5.4 - [Models](#models)
      - 5.4.1 - [Date & Genre](#date_genre)
      - 5.4.2 - [Genre Only](#genre)
      - 5.4.3 - [Date Only: Default](#date_default)
      - 5.4.4 - [Date Only: Reverse](#date_reverse)
      - 5.4.5 - [Date Sorted by Popularity](#date_popular)
    - 5.5 - [Best Model](#best_model)

In [ ]:
import os
os.mkdir('/content/collaborative')
print('Folder created!')
os.chdir('/content/collaborative')
print('Files are in this folder!')

Folder created!
Files are in this folder!


In [ ]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 13.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617551 sha256=fa5af511b5615988c8a96d9084c616bf1c45ca2bd311c2ec0b725ed155bbec5c
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
!python "MovieLens.py"
print('1 of 5: Done')
!python "RecommenderMetrics.py"
print('2 of 5: Done')
!python "EvaluationData.py"
print('3 of 5: Done')
!python "EvaluatedAlgorithm.py"
print('4 of 5: Done')
!python "Evaluator.py"
print('5 of 5: All Scripts Loaded!')

1 of 5: Done
2 of 5: Done
3 of 5: Done
4 of 5: Done
5 of 5: All Scripts Loaded!


Simple User CF NLargest

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  9 10:10:04 2018

@author: Frank
"""

from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
        
testSubject = '25'
k = 10

# Load our data set and compute the user similarity matrix
ml = MovieLens()
data = ml.loadMovieLensLatestSmall()

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': True
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

# Get top N similar users to our test subject
# (Alternate approach would be to select users up to some similarity threshold - try it!)
testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )

kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])
# kNeighbors = []
# for rating in similarUsers:
#     if rating[1] > 0.95:
#         kNeighbors.append(rating)

# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore
    
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break





Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Apollo 13 (1995) 5.0
Fugitive, The (1993) 4.2
Shawshank Redemption, The (1994) 4.0
Usual Suspects, The (1995) 3.8
Pulp Fiction (1994) 3.8
Dances with Wolves (1990) 3.8
Silence of the Lambs, The (1991) 3.8
Get Shorty (1995) 3.4000000000000004
Forrest Gump (1994) 3.4000000000000004
Braveheart (1995) 3.4
While You Were Sleeping (1995) 3.4


Similarity score above 0.95

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  9 10:10:04 2018

@author: Frank
"""

from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
        
testSubject = '25'
k = 10

# Load our data set and compute the user similarity matrix
ml = MovieLens()
data = ml.loadMovieLensLatestSmall()

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': True
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

# Get top N similar users to our test subject
# (Alternate approach would be to select users up to some similarity threshold - try it!)
testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )

#kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])
kNeighbors = []
for rating in similarUsers:
    if rating[1] > 0.95:
      kNeighbors.append(rating)

# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore
    
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break





Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Shawshank Redemption, The (1994) 238.38686300848957
Forrest Gump (1994) 231.32069152737125
Pulp Fiction (1994) 219.6932309720794
Silence of the Lambs, The (1991) 194.3156853253334
Braveheart (1995) 170.0890032282776
Star Wars: Episode V - The Empire Strikes Back (1980) 164.6353878628516
Fight Club (1999) 162.77453422665283
Terminator 2: Judgment Day (1991) 154.5471750759872
Jurassic Park (1993) 152.50145698451485
Star Wars: Episode VI - Return of the Jedi (1983) 151.57118036145837
Usual Suspects, The (1995) 147.02748575673678


***

Item Based Collaborative

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  9 10:10:04 2018

@author: Frank
"""

from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
        
testSubject = '25'
k = 10

ml = MovieLens()
data = ml.loadMovieLensLatestSmall()

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': False
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

testUserInnerID = trainSet.to_inner_uid(testSubject)

# Get the top K items we rated
testUserRatings = trainSet.ur[testUserInnerID]

kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])
# kNeighbors = []
# for rating in testUserRatings:
#     if rating[1] > 4.5:
#         kNeighbors.append(rating)

# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)
    
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break

    




Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Hideaway (1995) 10.0
Swan Princess, The (1994) 10.0
Lammbock (2001) 10.0
In China They Eat Dogs (I Kina spiser de hunde) (1999) 10.0
Crimson Rivers 2: Angels of the Apocalypse (RiviÃ¨res pourpres II - Les anges de l'apocalypse, Les) (2004) 10.0
Green Butchers, The (GrÃ¸nne slagtere, De) (2003) 10.0
Citizen X (1995) 10.0
Mesrine: Killer Instinct (L'instinct de mort) (2008) 10.0
White Ribbon, The (Das weiÃe Band) (2009) 10.0
MacGyver: Lost Treasure of Atlantis (1994) 10.0
Upside Down: The Creation Records Story (2010) 10.0


Item Based Collaborative Item greater than 4.5

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  9 10:10:04 2018

@author: Frank
"""

from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
        
testSubject = '25'
k = 10

ml = MovieLens()
data = ml.loadMovieLensLatestSmall()

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': False
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

testUserInnerID = trainSet.to_inner_uid(testSubject)

# Get the top K items we rated
testUserRatings = trainSet.ur[testUserInnerID]

#kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])
kNeighbors = []
for rating in testUserRatings:
    if rating[1] > 4.5:
        kNeighbors.append(rating)

# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)
    
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break

    




Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Sherlock - A Study in Pink (2010) 19.981834308394514
Harry Brown (2009) 19.945384578860136
Girl Who Played with Fire, The (Flickan som lekte med elden) (2009) 19.876432725449966
Hitchcock (2012) 19.86829337377438
Girl Who Kicked the Hornet's Nest, The (Luftslottet som sprÃ¤ngdes) (2009) 19.862199692727163
Gangster Squad (2013) 19.845524291892616
Lawless (2012) 19.79993794158126
Batman: The Dark Knight Returns, Part 1 (2012) 19.795539012479875
Layer Cake (2004) 19.776679778759515
Passengers (2016) 19.760795311454046
Batman: The Dark Knight Returns, Part 2 (2013) 19.72814649044438


***

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from surprise import KNNBasic
from surprise import KNNWithZScore
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# User-based KNN - cosine
UserKNNcosine = KNNBasic(sim_options = {'name': 'cosine', 'user_based': True})
evaluator.AddAlgorithm(UserKNNcosine, "User KNN cosine")

# User-based KNN - msd
UserKNNmsd = KNNBasic(sim_options = {'name': 'msd', 'user_based': True})
evaluator.AddAlgorithm(UserKNNmsd, "User KNN msd")

# User-based KNN - pearson
UserKNNpearson = KNNBasic(sim_options = {'name': 'pearson', 'user_based': True})
evaluator.AddAlgorithm(UserKNNpearson, "User KNN pearson")

# User-based KNN - pearson_baseline
UserKNNpb = KNNBasic(sim_options = {'name': 'pearson_baseline', 'user_based': True})
evaluator.AddAlgorithm(UserKNNpb, "User KNN pearson_basline")

# User-based KNNZScore 
UserKNNzscore = KNNWithZScore(sim_options = {'name': 'cosine', 'user_based': True})
evaluator.AddAlgorithm(UserKNNzscore, "User KNNZScore")

# User-based KNNMeans
UserKNNmeans = KNNWithMeans(sim_options = {'name': 'cosine', 'user_based': True})
evaluator.AddAlgorithm(UserKNNmeans, "User KNNMeans")

# User-based KNNBaseline
UserKNNbline = KNNBaseline(sim_options = {'name': 'cosine', 'user_based': True})
evaluator.AddAlgorithm(UserKNNbline, "User KNNZBaseline")

# Item-based KNN - cosine
ItemKNNcosine = KNNBasic(sim_options = {'name': 'cosine', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNcosine, "Item KNN cosine")

# Item-based KNN - msd
ItemKNNmsd = KNNBasic(sim_options = {'name': 'msd', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNmsd, "Item KNN msd")

# Item-based KNN - pearson
ItemKNNpearson = KNNBasic(sim_options = {'name': 'pearson', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNpearson, "Item KNN pearson")

# Item-based KNN - pearson_baseline
ItemKNNpb = KNNBasic(sim_options = {'name': 'pearson_baseline', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNpb, "Item KNN pearson_basline")

# Item-based KNNZScore 
ItemKNNzscore = KNNWithZScore(sim_options = {'name': 'cosine', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNzscore, "Item KNNZScore")

# Item-based KNNMeans
ItemKNNmeans = KNNWithMeans(sim_options = {'name': 'cosine', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNmeans, "Item KNNMeans")

# Item-based KNNBaseline
ItemKNNbline = KNNBaseline(sim_options = {'name': 'cosine', 'user_based': False})
evaluator.AddAlgorithm(ItemKNNbline, "Item KNNZBaseline")


# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  User KNN cosine ...
Evaluating accuracy...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating top-N with leave-one-out...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  User KNN msd ...
Evaluating accuracy...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating top-N with leave-one-out...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing hi